#### RAG Pipeline
Create Dataset

Retrieve and process about 10 article from wikipedia for a comprehencive view
of various aspects of space exploration

In [ ]:
!pip install -U langchain langchain_openai langchain_experimental scikit-learn

In [ ]:
import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
from google import genai
client = genai.Client(api_key=os.environ['GOOGLE_API_KEY'])

In [ ]:
''' https://platform.openai.com/docs/models for a list of models '''
from openai import OpenAI
import os
client = OpenAI(api_key=openai_api_key)
os.environ['OPENAI_API_KEY'] = openai_api_key

def set_environment():
  variable_dict = globals().items()
  for key, value in variable_dict:
    if 'API' in key or 'ID' in key:
      os.environ[key] = value
set_environment()

##### Pipeline Component #1

In [ ]:
''' collect the data '''
import requests, re
from bs4 import BeautifulSoup
urls = [
    "https://en.wikipedia.org/wiki/Space_exploration", "https://en.wikipedia.org/wiki/Apollo_program", "https://en.wikipedia.org/wiki/Hubble_Space_Telescope", "https://en.wikipedia.org/wiki/Marse_over", "https://en.wikipedia.org/wiki/International_Space_Station", "https://en.wikipedia.org/wiki/SpaceX", "https://en.wikipedia.org/wiki/Juno_(spacecraft)", "https://en.wikipedia.org/wiki/Voyager_program", "https://en.wikipedia.org/wiki/Galileo_(spacecraft)", "https://en.wikipedia.org/wiki/Kepler_Space_Telescope", "https://en.wikipedia.org/wiki/History_of_space_exploration", "https://en.wikipedia.org/wiki/Human_spaceflight", "https://en.wikipedia.org/wiki/Robotic_space_mission", "https://en.wikipedia.org/wiki/International_Space_Station", "https://en.wikipedia.org/wiki/Space_agencies", "https://en.wikipedia.org/wiki/Mars_exploration", "https://en.wikipedia.org/wiki/Space_colonization", "https://en.wikipedia.org/wiki/Space_telescope", "https://en.wikipedia.org/wiki/Astrobiology", "https://en.wikipedia.org/wiki/Private_spaceflight" ]

In [ ]:
''' prepare the data '''
def remove_references(content):
  content = re.sub(r'\[\d+\]', '', content)
  return content

def return_clean_text(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  content = soup.find('div', {'class': 'mw-parser-output'})
  for section_title in ['References', 'Bibliography', 'See also', 'External links']:
    section = content.find('span', text=section_title)
    if section:
      for sibling in section.parent.find_next_siblings():
        sibling.decompose()
      section.parent.decompose()
  text = content.get_text(separator=' ', strip=True)
  text = remove_references(text)
  return text

for idx, url in enumerate(urls):
  filename = f'llm_data{idx}.txt'
  with open(filename, 'w', encoding='utf-8') as fd:
      fd.write(return_clean_text(url))
      fd.write('\n\n')
  fd.close() # close it for continuous streaming

<ipython-input-5-2b6afff22475>:11: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  section = content.find('span', text=section_title)


In [ ]:
with open('llm_data3.txt', 'r', encoding='utf-8') as fd:
  lines = fd.readlines()
  for line in lines:
    print(line.strip())

Look for Marse over on one of Wikipedia's sister projects : Wiktionary (dictionary) Wikibooks (textbooks) Wikiquote (quotations) Wikisource (library) Wikiversity (learning resources) Commons (media) Wikivoyage (travel guide) Wikinews (news source) Wikidata (linked database) Wikispecies (species directory) Wikipedia does not have an article with this exact name. Please search for Marse over in Wikipedia to check for alternative titles or spellings. You need to log in or create an account and be autoconfirmed to create new articles. Alternatively, you can use the article wizard to submit a draft for review, or request a new article . Search for " Marse over " in existing articles. Look for pages within Wikipedia that link to this title . Other reasons this message may be displayed: If a page was recently created here, it may not be visible yet because of a delay in updating the database; wait a few minutes or try the purge function . Titles on Wikipedia are case sensitive except for the 

In [ ]:
with open('llm_data0.txt', 'r', encoding='utf-8') as fd:
  text = fd.read()
print(text)
# process it in chunks
chunk_size = 1000
chunked_text = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
len(chunked_text), chunked_text[6]

##### Pipeline Component #2

In [ ]:
''' storage and retrieval
files are normally batch as a stream of incoming documents
Now Vectorize it or configure to load automatically into vector store
'''
!pip install -U -q deeplake
import deeplake
from deeplake import Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 34.3 MB/s eta 0:00:00


In [ ]:
''' create or load a Deeplake Dataset '''
#ds_ = Dataset("/content", mode='w')
from deeplake import types

bulk_data = deeplake.create("/content")
inverted_index_DS = '/content/inverted_index'
ds_= deeplake.create(inverted_index_DS)

bulk_data = deeplake.open_read_only("/content")
inverted_index_DS = '/content/inverted_index'
ds_= deeplake.open(inverted_index_DS)

ds_, ds_.metadata, ds_.summary()

Dataset length: 0
Columns:




(<deeplake._deeplake.Dataset at 0x7a52a9d3d1b0>,
 None)

In [ ]:
''' deeplake with dataset '''
def embedding_function(texts, model="text-embedding-ada-002"):
   if isinstance(texts, str): texts = [texts]
   texts = [t.replace("\n", " ") for t in texts]

   return [data.embedding for data in \
           OpenAI().embeddings.create(input = texts, model=model).data]

texts = []
for i in range(20):
  filename = f'llm_data{i}.txt'
  with open(filename, 'r', encoding='utf-8') as fd:
    text = fd.read()
  fd.close()
  chunked_text = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
  texts.extend(chunked_text)

embeddings = embedding_function(texts, model="text-embedding-ada-002")
len(texts), len(embeddings)

In [ ]:
ds_.add_column("text", types.Text())
ds_.add_column("embeddings", types.Embedding())
ds_.commit()
ds_.summary()

Dataset length: 0
Columns:
  text      : text
  embeddings: embedding(0, clustered)




In [ ]:
ds_.append({"text": texts, "embeddings": embeddings})
ds_.commit()
ds_.summary()

Dataset length: 1064
Columns:
  text      : text
  embeddings: embedding(1536, clustered)


